In [1]:
import sounddevice as sd
print(sd.query_devices())


   0 sof-hda-dsp: - (hw:0,0), ALSA (2 in, 2 out)
   1 sof-hda-dsp: - (hw:0,3), ALSA (0 in, 2 out)
   2 sof-hda-dsp: - (hw:0,4), ALSA (0 in, 2 out)
   3 sof-hda-dsp: - (hw:0,5), ALSA (0 in, 2 out)
   4 sof-hda-dsp: - (hw:0,6), ALSA (2 in, 0 out)
   5 sof-hda-dsp: - (hw:0,7), ALSA (2 in, 0 out)
   6 sysdefault, ALSA (128 in, 128 out)
   7 samplerate, ALSA (128 in, 128 out)
   8 speexrate, ALSA (128 in, 128 out)
   9 pulse, ALSA (32 in, 32 out)
  10 upmix, ALSA (8 in, 8 out)
  11 vdownmix, ALSA (6 in, 6 out)
  12 dmix, ALSA (0 in, 2 out)
* 13 default, ALSA (32 in, 32 out)


In [3]:
import sounddevice as sd
from scipy.io.wavfile import write, read
import numpy as np

def get_working_devices():
    devices = sd.query_devices()
    input_devices = []
    output_devices = []

    for i, dev in enumerate(devices):
        if dev['max_input_channels'] > 0:
            try:
                sd.default.device = (i, None)
                sd.default.samplerate = int(dev['default_samplerate'])
                sd.default.channels = 1
                test = sd.rec(100, dtype='int16')  # Short test
                sd.wait()
                input_devices.append((i, dev['name'], int(dev['default_samplerate'])))
            except Exception as e:
                continue

        if dev['max_output_channels'] > 0:
            try:
                sd.default.device = (None, i)
                sd.default.samplerate = int(dev['default_samplerate'])
                dummy = np.zeros(100, dtype='int16')
                sd.play(dummy, blocking=True)
                output_devices.append((i, dev['name'], int(dev['default_samplerate'])))
            except Exception as e:
                continue

    return input_devices, output_devices


def record_and_play(input_devices, output_device):
    for device_id, name, sr in input_devices:
        filename = f"recording_device_{device_id}.wav"
        print(f"\n🎙️ Recording from: {name} (Device {device_id}, SR: {sr})")

        sd.default.device = (device_id, output_device[0])
        sd.default.samplerate = sr
        sd.default.channels = 1

        duration = 15  # seconds
        recording = sd.rec(int(duration * sr), dtype='int16')
        sd.wait()
        write(filename, sr, recording)
        print(f"✅ Saved to {filename}")

        print("🔊 Playing back...")
        _, data = read(filename)
        sd.play(data, sr)
        sd.wait()
        print("✅ Playback complete.")


# 🔍 Main
input_devices, output_devices = get_working_devices()

if not input_devices:
    print("❌ No working input devices found.")
elif not output_devices:
    print("❌ No working output devices found.")
else:
    # Pick the first working output device
    output_device = output_devices[0]
    print(f"\n🟢 Using output device: {output_device[1]} (Device {output_device[0]})")
    record_and_play(input_devices, output_device)


❌ No working output devices found.


In [1]:
def record_audio(filename="input.wav", duration=5, samplerate=16000, input_device=5, output_device=9):
    try:
        print(f"🎙️ Recording from device {input_device}...")
        sd.default.device = (input_device, output_device)
        sd.default.samplerate = samplerate
        sd.default.channels = 1

        recording = sd.rec(int(duration * samplerate), dtype='int16')
        sd.wait()

        write(filename, samplerate, recording)
        print(f"✅ Saved to {filename}")

        # Playback
        print("🔊 Playing back...")
        _, data = read(filename)
        sd.play(data, samplerate=samplerate)
        sd.wait()
        print("✅ Playback complete.")

    except Exception as e:
        print(f"❌ Error: {e}")

In [ ]:

input_device = 5  # Use a working input device (e.g., 'default')
output_device = 9  # Often same as input; set to correct output if needed
sample_rate = 16000
duration = int(input("Enter recording duration (seconds): "))
filename = f"recording_device_{input_device}.wav"

try:
    print(f"🎙️ Recording from device {input_device}...")
    sd.default.device = (input_device, output_device)
    sd.default.samplerate = sample_rate
    sd.default.channels = 1

    recording = sd.rec(int(duration * sample_rate), dtype='int16')
    sd.wait()

    write(filename, sample_rate, recording)
    print(f"✅ Saved to {filename}")

    # Read for playback
    _, data = read(filename)

    print("🔊 Playing back...")
    sd.play(data, samplerate=sample_rate)
    sd.wait()
    print("✅ Playback complete.")

except Exception as e:
    print(f"❌ Error: {e}")
